In [1]:
%%writefile bus_data_distribution.py

from mrjob.job import MRJob
from mrjob.protocol import RawValueProtocol
import sys

class BusDataDist(MRJob):
    OUTPUT_PROTOCOL = RawValueProtocol
    
    def mapper_init(self):
        self.intervals=set(map(str, list(range(0,24))))
        self.found_intervals=set()
        
        
    def mapper(self, key, line):
        if line != ';;;;;;;':
            time=line.split(';')[1]
            try:
                (interval,_,_)=time.split(':')
                interval = interval[1] if interval[0]=='0' else interval 
                self.found_intervals.add(interval)
                yield(interval, 1)
            except ValueError:
                pass
    
    # Handle hour intervals which didn't occur in the data.
    def mapper_final(self):
        missed_intervals=self.intervals.difference(self.found_intervals)
        for interval in missed_intervals:
            yield(interval, 0)
    
    def combiner(self, intr, cnt_itr):
        yield(intr, sum(cnt_itr))
               
    def reducer(self, intr, cnt_itr):
        yield(None, f'{intr};{sum(cnt_itr)}')
        
    
        
if __name__ == '__main__':
    BusDataDist.run()

Overwriting bus_data_distribution.py


In [2]:
%run bus_data_distribution.py busdatas.csv

No configs found; falling back on auto-configuration
No configs specified for inline runner
Creating temp directory /tmp/bus_data_distribution.tuoms.20191204.154753.795925
Running step 1 of 1...
job output is in /tmp/bus_data_distribution.tuoms.20191204.154753.795925/output
Streaming final output from /tmp/bus_data_distribution.tuoms.20191204.154753.795925/output...


16;110325
17;68919
6;0
7;0
2;1658
20;73257
4;1715
5;0
14;159825
15;30058
10;7842
11;55839
0;11146
1;0
23;11008
3;9169
21;107740
22;66809
12;63900
13;267964
18;228476
19;88894
8;130807
9;222687


Removing temp directory /tmp/bus_data_distribution.tuoms.20191204.154753.795925...
